## A notebook to get counts for the HRA KG paper

In [22]:
# import and install packages
%pip install hra-api-client requests

import hra_api_client
from hra_api_client.api import v1_api
import requests
from pprint import pprint
from pathlib import Path

Note: you may need to restart the kernel to use updated packages.


In [23]:
# Client configuration, the default API endpoint is https://apps.humanatlas.io/api
configuration = hra_api_client.Configuration(
    host="https://apps.humanatlas.io/api"
)

# Base Client Object
api_client = hra_api_client.ApiClient(configuration)

# Instance with which to call the primary /v1 routes from our API
api_instance = v1_api.V1Api(api_client)

## Count and size of landmarks

In [24]:
# download landmarks via SPARQL

query = '''
#+ summary: Landmark GLB download URLs
#+ description: Landmark GLB download URLs

PREFIX ccf: <http://purl.org/ccf/>
PREFIX HRA: <https://purl.humanatlas.io/collection/hra>

SELECT DISTINCT ?glb_url 

WHERE {
#  ?refOrgan ccf:has_reference_organ ?refOrgan .
  ?refOrgan ccf:has_object_reference [
      ccf:file_url ?glb_url ;
    ccf:file_subpath ?scene_node  
  ] .
  FILTER(strstarts(str(?refOrgan), "https://purl.humanatlas.io/landmark"))
}
ORDER BY ?ref_organ
'''
api_response = api_instance.sparql(query=query, format="application/json")
rows = api_response['results']['bindings']

urls = [obj['glb_url']['value']  for obj in rows]

# checkpprint(urls)

In [25]:
# check if download folder exists
folder_path = Path("downloaded-organs")

# Check if the folder exists and create it if not
folder_path.mkdir(parents=True, exist_ok=True)

# download GLBs
for url in urls:

  # get file name
  name = url.split('/')[len(url.split('/'))-1]
  
  # Path where the file will be saved
  file_name = folder_path / name

  output_file = f"{file_name}.glb"

  # Download the GLB file
  try:
      response = requests.get(url, stream=True)
      response.raise_for_status()  # Check for HTTP request errors
      with open(output_file, "wb") as file:
          for chunk in response.iter_content(chunk_size=8192):
              file.write(chunk)
      print(f"Downloaded GLB file to {output_file}")
  except requests.exceptions.RequestException as e:
      print(f"Failed to download file: {e}")

Downloaded GLB file to downloaded-organs\3d-vh-f-uterus-landmark.glb.glb
Downloaded GLB file to downloaded-organs\VH_M_Urinary_Bladder_Landmarks.glb.glb
Downloaded GLB file to downloaded-organs\3d-vh-f-urinary-bladder-landmark.glb.glb
Downloaded GLB file to downloaded-organs\VH_M_Ureter_Landmarks.glb.glb
Downloaded GLB file to downloaded-organs\3d-vh-f-ureter-r-landmark.glb.glb
Downloaded GLB file to downloaded-organs\3d-vh-f-ureter-l-landmark.glb.glb
Downloaded GLB file to downloaded-organs\3d-vh-m-trachea-landmark.glb.glb
Downloaded GLB file to downloaded-organs\3d-vh-f-trachea-landmark.glb.glb
Downloaded GLB file to downloaded-organs\VH_M_Spleen_Landmarks.glb.glb
Downloaded GLB file to downloaded-organs\VH_M_Spleen_Extraction_Sites.glb.glb
Downloaded GLB file to downloaded-organs\3d-vh-f-spleen-landmark.glb.glb
Downloaded GLB file to downloaded-organs\3d-vh-f-spleen-extraction.glb.glb
Downloaded GLB file to downloaded-organs\VH_M_Spinal_Cord_Landmarks.glb.glb
Downloaded GLB file to 